In [11]:
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.chrome.options import Options
import re
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import urllib
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import time
import datetime

driver_path='C:\Program Files (x86)\Google\Chrome\chromedriver78.exe'
def init_driver():
    driver = webdriver.Chrome(driver_path)  #need to specify location of driver
    driver.wait = WebDriverWait(driver,3)
    return driver

def get_driver(driver,URLInitial):
    url=URLInitial
    driver.get(url)
    
def is_visible_by_link_text(locator_text, timeout = 30): 
    try: 
        ui.WebDriverWait(chrome, timeout).until(EC.visibility_of_element_located((By.XPATH, locator))) 
        return True
    except TimeoutException: 
        return False

def init_driver_Headless():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920x1080")
    driver = webdriver.Chrome(chrome_options= chrome_options  ,executable_path=driver_path)  #make dynamic
    driver.wait = WebDriverWait(driver,3)
    return driver


#taskkill /F /IM chromedriver.exe ---- kill chrome driver if testing around and multiple instances are created

In [20]:
# if the chrome driver used does not match the version of chrome that you have, download 
# the chromedriver that matches yur version of chrome
try:
    driver=init_driver()
    driver.set_page_load_timeout(30)
    get_driver(driver,"https://www.realclearpolitics.com/epolls/2020/president/us/2020_democratic_presidential_nomination-6730.html")
    driver.set_page_load_timeout(10)
    try: #catch timeout
        try: #catch element exception
            driver.find_element_by_link_text("I Agree").click() 
            time.sleep(1)
            driver.find_element_by_link_text("More Polling Data").click()
        except NoSuchElementException:
            driver.find_element_by_link_text("More Polling Data").click()
        time.sleep(2)
        page_source=driver.page_source
    except TimeoutException as ex:
        driver.execute_script("window.stop();")
        page_source=driver.page_source
    soup=bs(page_source)
    driver.quit()
    print('No Errors!')
except Exception as e:
    print(e)
    driver.quit()
    print('Driver exited on unkown error')

soup2=soup.find("div", { "id" : "polling-data-full" })
soup3=soup2.find("table", { "class" : "data large" })
dfs = pd.read_html(str(soup3))[0]

#driver.find_element_by_link_text("More Polling Data").click()

Message: element click intercepted: Element <a id="more_table_data" href="javascript:void('0');">...</a> is not clickable at point (455, 880). Other element would receive the click: <div role="dialog" aria-live="polite" aria-label="cookieconsent" aria-describedby="cookieconsent:desc" class="cc-window cc-banner cc-type-info cc-theme-block cc-bottom cc-color-override-1456576595 " style="">...</div>
  (Session info: chrome=78.0.3904.70)

Driver exited on unkown error


NameError: name 'soup' is not defined

In [16]:
page_source

NameError: name 'page_source' is not defined

In [6]:
import re
pattern = '\d{1,2}/\d{1,2}'#' - \d{1,2}/\d{1,2}'
dfs["clean_start_year"]=""
dfs["clean_end_year"]=""
dfs["start_month"]=""
dfs["end_month"]=""
dfs["start_day"]=""
dfs["end_day"]=""
now = datetime.datetime.now()

for i,row in dfs.iterrows():
    result = re.findall(pattern, row.Date)
    dfs["start_month"].iloc[i]=int(result[0][:result[0].find('/')])
    dfs["end_month"].iloc[i]=int(result[1][:result[1].find('/')])
    dfs["start_day"].iloc[i]=int(result[0][result[0].find('/')+1:])
    dfs["end_day"].iloc[i]=int(result[1][result[1].find('/')+1:])
    if i==0:
        last_year_start=now.year
        last_year_end=now.year
        dfs["clean_start_year"]=now.year
        dfs["clean_end_year"]=now.year
        start_last_month_temp=dfs["start_month"].iloc[i]
        end_last_month_temp=dfs["start_month"].iloc[i]
    
    if start_last_month_temp< dfs["start_month"].iloc[i]: last_year_start=last_year_start-1
    if end_last_month_temp< dfs["end_month"].iloc[i] : last_year_end=last_year_end-1 

    dfs["clean_start_year"].iloc[i]= last_year_start
    dfs["clean_end_year"].iloc[i]= last_year_end
    
    start_last_month_temp=dfs["start_month"].iloc[i]
    end_last_month_temp=dfs["end_month"].iloc[i]
dfs['start_date']=datetime.datetime.now()
dfs['end_date']=datetime.datetime.now()
dfs['start_date']=dfs.apply(lambda x:datetime.datetime(x.clean_start_year, x.start_month, x.start_day),axis=1 )
dfs['end_date']=dfs.apply(lambda x:datetime.datetime(x.clean_end_year, x.end_month, x.end_day),axis=1 )

C:\Users\keval\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
dfs.head()
dfs=dfs.drop(['Date','Spread','clean_start_year','clean_end_year','start_month','end_month','start_day','end_day'],axis=1)

In [8]:
dfs.set_index(['Poll','start_date','end_date'],inplace=True)

In [10]:
dfs.replace("--",0,inplace=True)
dfs=dfs.apply(pd.to_numeric)
dfs

,,,Biden,Sanders,Warren,Harris,Buttigieg,O'Rourke,Yang,Booker,Castro,Klobuchar,Gabbard,Delaney,Steyer
Poll,start_date,end_date,,,,,,,,,,,,,
RCP Average,2019-07-21,2019-07-30,32.2,16.2,14.3,10.8,5.5,2.8,1.8,1.7,1.0,1.0,0.8,0.8,0.8
Economist/YouGovEconomist,2019-07-27,2019-07-30,26.0,13.0,20.0,11.0,6.0,2.0,1.0,3.0,2.0,0.0,2.0,1.0,0.0
EmersonEmerson,2019-07-27,2019-07-29,33.0,20.0,14.0,11.0,6.0,4.0,2.0,0.0,1.0,0.0,1.0,1.0,2.0
The Hill/HarrisXThe Hill,2019-07-27,2019-07-28,34.0,20.0,12.0,9.0,5.0,4.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
QuinnipiacQuinnipiac,2019-07-25,2019-07-28,34.0,11.0,15.0,12.0,6.0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0
Politico/Morning ConsultPolitico,2019-07-22,2019-07-28,33.0,18.0,13.0,12.0,5.0,3.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0
FOX NewsFOX News,2019-07-21,2019-07-23,33.0,15.0,12.0,10.0,5.0,2.0,3.0,2.0,1.0,3.0,0.0,1.0,1.0
Economist/YouGovEconomist,2019-07-21,2019-07-23,25.0,13.0,18.0,9.0,7.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,1.0
Politico/Morning ConsultPolitico,2019-07-15,2019-07-21,33.0,18.0,14.0,13.0,5.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0


In [502]:
min_date=min(dfs.index.levels[1])
max_date=max(dfs.index.levels[2])
newdf=pd.DataFrame(columns=dfs.columns,index=pd.date_range(start=min_date, end=max_date))
k=0
for i,row in newdf.iterrows():
    tempseries=row.fillna(0)
    k=0
    for j,row2 in dfs.iterrows():
        if j[1] <= i<=j[2] :
            k+=1
            tempseries=row2.add(tempseries)
    newdf.loc[i]=tempseries/k
newdf.fillna(method='ffill',inplace=True)
newdf.to_csv("RCP Altered Poll Data.csv",index_label='Date')

In [13]:
newdf=pd.read_csv("RCP Altered Poll Data.csv",index_col='Date')